In [1]:
import h5py
import numpy as np
import math
from sklearn.preprocessing import normalize
import pandas as pd
import Preprocessing #my module
from Preprocessing import *

In [2]:
dataset = h5py.File("/home/roberts/O-PPAC/simu_HDF_thistimeforReal.h5", 'r') #load data
num_diodes = dataset['Num_diodes'][...] #separate the data into each key within the data file
x_pos = (dataset['Xpos'][...])/100 #puts the scale into mm
y_pos = (dataset['Ypos'][...])/100
histgrid = dataset['histgrid'][...]
histgrid = histgrid[:-10,:,:] #redefine the shape and size of histgrid by removing the last 10 events (they are bad)
output = np.vstack((x_pos,y_pos)) #define new, single variable for x and y position
output = np.transpose(output) #original definition was sideways, so we transposed it

print(histgrid.shape)

dataset.close()

(991, 342, 132)


In [3]:
#make sure there are no NaNs (FCNN gets angry when NaNs)
assert not(np.any(np.isnan(histgrid)))

In [4]:
all_data, all_labels = data_compile(histgrid, output)

#to make sure everything is the correct shape
print(all_data.shape)
print(all_labels.shape)

(338922, 132)
(338922, 2)


In [5]:
all_data = normalize(all_data,axis = 1)
print(np.max(all_data))

0.8395246220687987


In [ ]:
#define all_labels and all_data as df's to better view rows and add column
all_labels = pd.DataFrame(all_labels)
all_data = pd.DataFrame(all_data)
all_data["sum"] = np.sum(all_data, axis = 1) #column added to find rows with only 0's

#confirming number of initial entries
print(len(all_labels))
print(len(all_data))

#find all the locations in all_data (by index) where the only values are 0 across the diodes
remove_labels = list(all_data[all_data["sum"] == 0].index)
print(len(remove_labels))

#added these to ensure I have unshuffled versions of the data and labels for finding locations with larger error
unshuf_labels = all_labels.drop(all_labels.index[remove_labels])
unshuf_data = all_data[all_data["sum"] != 0]

#lets all_labels and all_data start with the same indices and values as their unshuffled counterparts
all_labels = unshuf_labels
unshuf_labels = unshuf_labels.to_numpy() #dfs are hard to graph with, so I converted to np.arrays for future use
all_data = unshuf_data
unshuf_data = unshuf_data.to_numpy()

#confirm new number of entries (should be len(remove_labels) less than the intital value)
print(len(all_labels))
print(len(all_data))

#must remove the sum column before using in FCNN; otherwise it will treat it as a data point 
del all_data['sum']

In [ ]:
dataset = np.concatenate((all_data, all_labels), axis=1)
#puts the x,y labels at the end of each data row

In [ ]:
print(dataset.shape)  #confirm it is the correct shape

In [ ]:
np.save("recomplied_dataset_fixed" , dataset)

In [ ]:
#making sure it works
new_dataset = np.load("/home/roberts/O-PPAC/recomplied_dataset_fixed.npy")

In [ ]:
print(new_dataset.shape)